# Combining our Post-Starburst Regions and the BPT Diagrams

With both sets of data now available to us, let's examine for any correlations between Post-Starburst, and classifications under the BPT model. We will do so by examining the percent of Post-Starburst regions within each classification.

For example:
- How much of Star-Formation dominated regions are also Post-Starburst regions?
- How much of LINER dominated regions are also Post-Starburst regions?

## Methodology

Since we choose to create a mask to identify our PSB regions, comparing them with the BPT masks will be very straightforward!

Let's first reintroduce our PSB mask method from earlier:

In [2]:
from marvin.tools import Maps
import marvin
import numpy as np
import matplotlib.pyplot as plt

[WARNING]: cannot initiate Sentry error reporting: [Errno 25] Inappropriate ioctl for device. (UserWarning)
[INFO]: No release version set. Setting default to DR17
[WARNING]: path /home/neap/sas/dr17/manga/spectro/analysis/v3_1_1/3.1.0/dapall-v3_1_1-3.1.0.fits cannot be found. Setting dapall to None. (MarvinUserWarning)


In [3]:
def mask_PSB(hvalue, dvalue, divar, snr): 
    psb = np.zeros(hvalue.shape, dtype=bool)

    var = 1 / np.sqrt(divar)
    psb[np.logical_and(np.logical_and(hvalue < 3, (dvalue - var) > 4), snr > 3)] = True

    return psb

Let's expand on this function, and implement it into a parent function that will compare the PSB regions and BPT classifications of any galaxy. 
First, we'll need to retrieve the spectra values for any galaxy, which we can then input into our masks.

In [4]:
def get_histogram(galaxy_id):
    np.seterr(divide='ignore')
    galaxy = marvin.tools.Maps(galaxy_id, bintype='HYB10')
    alpha = galaxy.emline_gew_ha_6564
    delta = galaxy.specindex_hdeltaagalaxy
    snr = galaxy.spx_snr
    snr_value = getattr(snr, 'value', None)
    value = getattr(alpha, 'value', None)
    dvalue = getattr(delta, 'value', None)
    divar = getattr(delta, 'ivar', None)
    maskpsb = mask_PSB(value, dvalue, divar, snr_value)

Let's also generate the BPT masks for this galaxy as well!

In [ ]:
masks, fig, axes = galaxy.get_bpt(use_oi = False, show_plot=True)

If you have been following close attention to both our PSB and the BPT masks, you will notice a small issue. Marvin generation of BPT diagrams does not quite follow the same specification as our PSB masks. In particular, the BPT diagrams impose a SNR cut of 10, while we are much more lenient and allow a SNR over 3. Part of this is due to generating an adequate sample size. Through visual testing and analysis, we observed that a SNR cut of 3 would only leave trace amounts of spurious values, while optimizing the number of selected PSB spaxels.

This now poses the issue that a large number of our PSB spaxels will be unclassified by the BPT diagram, so lets create a new function that will help us identify how many unclassified spaxels exist. 

To keep the logic simple, our new function will return the total spaxels in our PSB mask, which we can use like so:

Total Spaxels - Classified Spaxels = Unclassified Spaxels

In [5]:
def get_total_spaxels(dapmap):
    value = getattr(dapmap, 'value', None)
    ivar = getattr(dapmap, 'ivar', None)
    low_snr = mask_low_snr(value, ivar, 3)
    nocov = dapmap.pixmask.get_mask('NOCOV') 
    good_spax = np.ma.array(value, mask=np.logical_or.reduce((nocov, low_snr))) #This method is most similar to how marvin uses masks 
                                                                                #to cover unwanted spaxels, but there are many ways to do this.
    valid_counts = good_spax.count()
    return valid_counts

def mask_low_snr(value, ivar, snr_min):

    low_snr = np.zeros(value.shape, dtype=bool)

    if (ivar is not None) and (not np.all(np.isnan(ivar))):
        low_snr = (ivar == 0.)

        if snr_min is not None:
            low_snr[np.abs(value * np.sqrt(ivar)) < snr_min] = True

    return low_snr

Now, let's combine everything into our original function!

In [6]:
def get_histogram(galaxy_id):
    np.seterr(divide='ignore')
    galaxy = marvin.tools.Maps(galaxy_id, bintype='HYB10')
    alpha = galaxy.emline_gew_ha_6564
    delta = galaxy.specindex_hdeltaagalaxy
    snr = galaxy.spx_snr
    snr_value = getattr(snr, 'value', None)
    
    value = getattr(alpha, 'value', None)
    dvalue = getattr(delta, 'value', None)
    divar = getattr(delta, 'ivar', None)
    
    maskpsb = mask_PSB(value, dvalue, divar, snr_value)
    total_spax = get_total_spaxels(alpha)

    masks, fig, axes = galaxy.get_bpt(use_oi = False, show_plot=False)

Great! Now that we have both masks in this function, let's combine them. Using masking properties, combining two masks will give us a new mask that only contains indices where both masks are true.

For example:
- Combining Star-Formation mask with PSB-mask:

SF_mask is the new mask, where only indices with both PSB = True, and SF = True are marked. This is equivalent to telling us the spaxels where a galaxy is both Star-Forming and Post-Starburst (which we would expect to be rare).

With this mask, we can now count the amount of Star-Forming spaxels that are PSB, as well as Non-PSB 

We repeat this process for all 5 classifications in the BPT model, as well as the unclassified spaxels.

In [ ]:
sf_mask = maskpsb & masks['sf']['nii'] 
sf_psbcount = np.count_nonzero(sf_mask)
sf_nopsb = np.count_nonzero(masks['sf']['nii']) - sf_psbcount

In [7]:
def get_histogram(galaxy_id):
    np.seterr(divide='ignore')
    galaxy = marvin.tools.Maps(galaxy_id, bintype='HYB10')
    alpha = galaxy.emline_gew_ha_6564
    delta = galaxy.specindex_hdeltaagalaxy
    snr = galaxy.spx_snr
    snr_value = getattr(snr, 'value', None)
    
    value = getattr(alpha, 'value', None)
    dvalue = getattr(delta, 'value', None)
    divar = getattr(delta, 'ivar', None)
    
    maskpsb = mask_PSB(value, dvalue, divar, snr_value)
    total_spax = get_total_spaxels(alpha)
    #print(np.count_nonzero(maskpsb))

    masks, fig, axes = galaxy.get_bpt(use_oi = False, show_plot=False)

    comp_mask = maskpsb & masks['comp']['nii'] 
    comp_psbcount = np.count_nonzero(comp_mask)
    comp_nopsb = np.count_nonzero(masks['comp']['nii']) - comp_psbcount

    sf_mask = maskpsb & masks['sf']['nii'] 
    sf_psbcount = np.count_nonzero(sf_mask)
    sf_nopsb = np.count_nonzero(masks['sf']['nii']) - sf_psbcount

    amb_mask = maskpsb & masks['ambiguous']['global'] 
    amb_psbcount = np.count_nonzero(amb_mask)
    amb_nopsb = np.count_nonzero(masks['ambiguous']['global']) - amb_psbcount

    seyf_mask = maskpsb & masks['seyfert']['sii'] 
    seyf_psbcount = np.count_nonzero(seyf_mask)
    seyf_nopsb = np.count_nonzero(masks['seyfert']['sii']) - seyf_psbcount

    liner_mask = maskpsb & masks['liner']['sii'] 
    liner_psbcount = np.count_nonzero(liner_mask)
    liner_nopsb = np.count_nonzero(masks['liner']['sii']) - liner_psbcount

    psb_noncategorized = np.count_nonzero(maskpsb) - comp_psbcount - sf_psbcount - amb_psbcount - seyf_psbcount - liner_psbcount
    nonpsb_noncategorized = np.count_nonzero(masks['invalid']['global'])

    return [comp_psbcount, comp_nopsb, sf_psbcount, sf_nopsb, amb_psbcount, amb_nopsb, seyf_psbcount, seyf_nopsb, liner_psbcount, liner_nopsb, psb_noncategorized, nonpsb_noncategorized]

Finally, let's plot all these numbers into a histogram so we can visually analyze what is going on.

In [8]:
def plot(galaxy):
    counts = get_histogram(galaxy)
    comp_psbTotal += counts[0]
    comp_nopsbTotal += counts[1]
    sf_psbTotal += counts[2]
    sf_nopsbTotal += counts[3]
    amb_psbTotal += counts[4]
    amb_noTotal += counts[5]
    seyf_psbTotal += counts[6]
    seyf_noTotal += counts[7]
    liner_psbTotal += counts[8]
    liner_nopsbTotal += counts[9]
    psb_noncategorized += counts[10]
    nopsb_noncategorized += counts[11]

    categories = ('Star Forming', 'Composite', 'Ambiguous', 'Seyfert', 'LINER', 'Nonclassified')
    category_counts = {
    'PSB': (sf_psbTotal, comp_psbTotal, amb_psbTotal, seyf_psbTotal, liner_psbTotal, psb_noncategorized),
    'No PSB': (sf_nopsbTotal, comp_nopsbTotal, amb_noTotal, seyf_noTotal, liner_nopsbTotal, nopsb_noncategorized),
    }

    x = np.arange(len(categories))  # the label locations
    width = 0.4
    multiplier = 0

    fig, ax = plt.subplots(layout='constrained')

    for category, count in category_counts.items():
        offset = width * multiplier
        rects = ax.bar(x + offset, count, width, label=category)
        ax.bar_label(rects, padding=3)
        multiplier += 1

    ax.set_ylabel('Frequency')
    ax.set_title('PSB vs non-PSB Spaxels for all galaxies')
    ax.set_xticks(x + width - 0.20, categories)
    ax.legend(loc='upper left')
    maxy = max([sf_psbTotal, comp_psbTotal, amb_psbTotal, seyf_psbTotal, liner_psbTotal, sf_nopsbTotal, comp_nopsbTotal, amb_noTotal, seyf_noTotal, liner_nopsbTotal])
    maxy = 1.2*maxy
    ax.set_ylim([0, maxy])
    ax.text(0.87, 0.76, str(nopsb_noncategorized),
        horizontalalignment='right',
        verticalalignment='top',
        transform = ax.transAxes)

    plt.show()

Let's look at some results in the next page!